In [1]:
import torch
import re
import sys
import collections
import string

import numpy as np
import pandas as pd

from multiprocessing import cpu_count
from nusacrowd import NusantaraConfigHelper
from datetime import datetime
from tqdm import tqdm
from huggingface_hub import HfApi, create_repo

from datasets import (
    load_dataset, 
    Dataset,
    DatasetDict
)
from transformers import (
    pipeline,
    AutoModelForQuestionAnswering, 
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    AutoModelForSequenceClassification, 
    T5ForConditionalGeneration, 
    T5Tokenizer
)

In [20]:
MODEL_QA_NAME = "muhammadravi251001/fine-tuned-DatasetQAS-IDK-MRC-with-xlm-roberta-large-without-ITTL-without-freeze-LR-1e-05"

tokenizer_kwargs = {'truncation': True, 'max_length': 512}

tokenizer_qa = AutoTokenizer.from_pretrained(MODEL_QA_NAME)
model_qa = AutoModelForQuestionAnswering.from_pretrained(MODEL_QA_NAME)
model_qa.config.top_k = 3
#model_qa = model_qa.to(device)

In [ ]:
def nlp_qa_original(question, context):
        
    inputs = tokenizer_qa(question, context, 
                          return_tensors="pt",
                          **tokenizer_kwargs)

    #inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model_qa(**inputs)

    sorted_start_logits = torch.argsort(outputs.start_logits)
    sorted_end_logits = torch.argsort(outputs.end_logits)

    answer_array = []
    for i in range(1, (top_k + 1)):

        start_index = sorted_start_logits[0, -i]
        end_index = sorted_end_logits[0, -i]
        
        print(start_index)
        print(end_index)
        
        answer_tokens = inputs["input_ids"][0][start_index : end_index + 1]

        answer = tokenizer_qa.decode(answer_tokens)
        answer_array.append({'answer': answer})

    return answer_array

In [36]:
def nlp_qa(question, context, top_k=5):
        
    inputs = tokenizer_qa(question, context, 
                          return_tensors="pt",
                          **tokenizer_kwargs)

    #inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model_qa(**inputs)

    sorted_start_logits = torch.argsort(outputs.start_logits)
    sorted_end_logits = torch.argsort(outputs.end_logits)
    
    print(outputs.start_logits.shape)

    answer_array = []
    for i in range(1, (top_k + 1)):

        start_index = sorted_start_logits[0, -i]
        end_index = sorted_end_logits[0, -i]
        
        print(outputs.start_logits[0, start_index])
        print(outputs.end_logits[0, end_index])
        print()
        
        #print(start_index)
        #print(end_index)
        
        answer_tokens = inputs["input_ids"][0][start_index : end_index + 1]

        answer = tokenizer_qa.decode(answer_tokens)
        answer_array.append({'answer': answer})

    return answer_array

In [18]:
context = "Protista adalah mikroorganisme eukariota yang bukan hewan, tumbuhan, atau fungus. Mereka pernah dikelompokkan ke dalam satu kerajaan bernama Protista, namun sekarang tidak dipertahankan lagi.[2] Penggunaannya masih digunakan untuk kepentingan kajian ekologi dan morfologi bagi semua organisme eukariotik bersel tunggal yang hidup secara mandiri atau, jika membentuk koloni, bersama-sama namun tidak menunjukkan diferensiasi menjadi jaringan yang berbeda-beda.[3]. Dari sudut pandang taksonomi, pengelompokan ini ditinggalkan karena bersifat parafiletik. Organisme dalam Protista tidak memiliki kesamaan, kecuali pengelompokan yang mudah[4]—baik yang bersel satu atau bersel banyak tanpa memiliki jaringan. Protista hidup di hampir semua lingkungan yang mengandung air. Banyak protista, seperti algae, adalah fotosintetik dan produsen primer vital dalam ekosistem, khususnya di laut sebagai bagian dari plankton. Protista lain, seperti Kinetoplastid dan Apicomplexa, adalah penyakit berbahaya bagi manusia, seperti malaria dan tripanosomiasis."

In [19]:
question = "Mengapa protista tidak dikelompokkan ke dalam satu kerajaan taksonomi?"

In [37]:
nlp_qa(question=question, context=context)

torch.Size([1, 228])
tensor(7.8845, grad_fn=<SelectBackward0>)
tensor(8.2361, grad_fn=<SelectBackward0>)

tensor(6.0922, grad_fn=<SelectBackward0>)
tensor(7.1699, grad_fn=<SelectBackward0>)

tensor(5.7972, grad_fn=<SelectBackward0>)
tensor(5.7037, grad_fn=<SelectBackward0>)

tensor(5.6187, grad_fn=<SelectBackward0>)
tensor(5.1508, grad_fn=<SelectBackward0>)

tensor(5.5576, grad_fn=<SelectBackward0>)
tensor(5.0051, grad_fn=<SelectBackward0>)



[{'answer': ''},
 {'answer': 'karena bersifat parafiletik'},
 {'answer': ''},
 {'answer': ''},
 {'answer': ''}]